In [58]:
from groq import Groq
import yaml
import PyPDF2
import ast
import json
import requests
import time


In [2]:
with open('env_variables.yml', 'r') as file:
    env_variables = yaml.safe_load(file)

In [138]:

# URL del API
url = "https://us-central1-ferneutron-demos.cloudfunctions.net/alpacademy-retriever"

# Realizar la solicitud GET
def get_rag(tema):
    output=None
    response = requests.post("https://us-central1-ferneutron-demos.cloudfunctions.net/alpacademy-retriever", json={"query": tema})
    try:
        
        output= json.loads(response.content)
    except:
        time.sleep(2)
        output= json.loads(response.content)

    if output!=None:
        
        output="\n".join(i['text'][:3] for i in output['results'])
        return output
    else:
        return None
        


In [139]:
client = Groq(api_key=env_variables['GROQ_API_KEY'])


In [140]:
with open('lessons.json', 'r') as file:
    plan = json.load(file)

In [156]:
def call_llama(titulo, retrival):
    # Crear el mensaje para el modelo
    completion = client.chat.completions.create(
        model="llama-3.1-70b-versatile",
        messages=[
            {
                "role": "system",
                "content": """Eres un profesor para sexto de primaria de méxico. Tu tarea es generar la clase, tal que el alumno lea bullets de información con los cuales aprende el tema que te solicita el usuario, debes tener en cuenta
                el contenido del cual hablaremos, que te proporcionaremos mediante un resumen que deberás usar para explicar el tema.

                Recuerda que tu usuario es un niño de 12 años que necesita la información clave y concisa.
                Para ayudarte te compartiremos un fragmento del libro el cual usarás para extraer la información relevante para dar la clase. El output que generes ya debe estar listo para que un alumno lo lea, por lo que debes esforzarte en ser didáctico.

                Entregaras un set de 3 clases. en forma de lista separados el sigueitne caracter [---]
                Para cada clase da 6 bullets importantes para aprender del tema
                Por ejemplo:

                Titulo clase y tu abstracto de informacion para aprender
                Estructura clase 2 
                Estructura clase 3





                No des ningun texto adicional. Regresa la repsuesta OBLIGATORIAMENTE en el formato especificado. No agregues mas texto.

                No me expliques que debo aprender. Recuerda que tu respuesta será leida por el alumno, sin ninun tratamiento adicional. sólo indica el texto a enseñar

                Cada elemento de la lista debe ser una enseñanza de a lo mas 500 caracteres
                """
            },
            {
                "role": "user",
                "content": f"""Este es el capítulo a enseñar** {titulo} **
                
                Este es el contenido del libro que te ayudará a seleccionar lo relevante para la clase:
                
                
                {retrival}."""
                }
        ],
        temperature=1.2,
        max_tokens=8000,
        top_p=1,
        stream=False,
        stop=None,
        )
    return completion.choices[0].message.content


In [162]:
materiales={}
for semana in plan.keys():
    titulo=plan[semana]['Contenido']
    aprendizajes= plan[semana]['Aprendizajes clave']
    to_rag=titulo + ' ' + ' '.join(aprendizajes)
    moc=call_llama(titulo, to_rag)

    moc=moc.split("---")


    moc=[i for i in moc if len(i)>=20]

    materiales[semana]={'material':moc,
                        'tipo':['clase']*len(moc)}

In [171]:
materiales['Semana 1']['material']

['**La Era de la Prehistoria**\n\n* La prehistoria es el período de tiempo que comprende desde la aparición del primer ser humano hasta la invención de la escritura.\n* Durante la prehistoria, los seres humanos se desarrollaron y se extendieron por todo el mundo, lo que se conoce como el poblamiento de los continentes.\n* La prehistoria se divide en tres grandes periodos: Paleolítico (hace 2,5 millones de años hasta hace 10.000 años), Mesolítico (hace 10.000 años hasta hace 5.000 años) y Neolítico (hace 5.000 años hasta hace 2.000 años).\n* Durante la prehistoria, los seres humanos se dedicaron a la caza y la recolección de alimentos, hasta que se desarrolló la agricultura y la ganadería.\n* La domesticación de plantas y animales permitió a los seres humanos establecerse en un lugar y formar comunidades más grandes.\n* La prehistoria nos deja un legado en forma de herramientas de piedra, pinturas rupestres y otros objetos que nos ayudan a entender cómo vivían nuestros antepasados.\n\n'

In [178]:
def call_llama_qa(compendio):
    # Crear el mensaje para el modelo
    completion = client.chat.completions.create(
        model="llama-3.1-70b-versatile",
        messages=[
            {
                "role": "system",
                "content": """Eres un profesor de sexto de primaria de méxico. Recibirás un texto, y debes generar 3 preguntas con su respuesta para que podamos hacer un examen del tema visto.
                No des tu opinion limitate a dar tu respuesta en el siguiente formato

                Pregunta1
                respuesta 1

                pregunta 2
                respeusta 2

                pregunta 3
                respeusta 3
                """
            },
            {
                "role": "user",
                "content": f"""Este es el material visto en clase: ** {compendio} **
                """
                }
        ],
        temperature=1.2,
        max_tokens=8000,
        top_p=1,
        stream=False,
        stop=None,
        )
    return completion.choices[0].message.content


In [186]:
for i in materiales.keys():

    questions=call_llama_qa(materiales[i]['material'])

    materiales[i]['material'].append(questions)
    materiales[i]['tipo'].append("preguntas")
    

In [187]:
def call_llama_examen(compendio):
    # Crear el mensaje para el modelo
    completion = client.chat.completions.create(
        model="llama-3.1-70b-versatile",
        messages=[
            {
                "role": "system",
                "content": """Eres un profesor de sexto de primaria de méxico. Recibirás un texto, y debes generar un examen de 10 preguntas respecto a lso temas vistos y a las pregutnas que se les realizaron de reforzamiento.

                No des tu opinion limitate a dar tu respuesta en el siguiente formato

                Pregunta1
                respuesta 1

                pregunta 2
                respeusta 2

                pregunta 3
                respeusta 3
                
                """
            },
            {
                "role": "user",
                "content": f"""Este es el material visto en clase: ** {compendio} **
                """
                }
        ],
        temperature=1.2,
        max_tokens=8000,
        top_p=1,
        stream=False,
        stop=None,
        )
    return completion.choices[0].message.content


In [193]:
for i in materiales.keys():

    questions=call_llama_examen(materiales[i]['material'])

    materiales[i]['material'].append(questions)
    materiales[i]['tipo'].append("examen")


In [196]:
with open('../estados_materiales.json', 'w') as fp:
    json.dump(materiales, fp)